In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from plot_functions import *

# Pie Chart Sample Data
pie_data = {
    '2023': {
        'counts':[40, 30, 20],
        'crime_count':100
    },
    '2024': {
        'counts':[50, 35, 25],
        'crime_count':20
    }
}

In [8]:
import requests
API_BASE_URL = "http://localhost:9090/api"

def get_suburbs():
    response = requests.get(f"{API_BASE_URL}/suburbs")
    return response.json() if response.status_code == 200 else {}

def get_crime_weather_data(suburb):
    response = requests.get(f"{API_BASE_URL}/{suburb}/crime-weather")
    return response.json() if response.status_code == 200 else {}

def get_suburbs_with_weather():
    all_suburbs = []
    all_suburbs_data = get_suburbs()
    for suburb_data in all_suburbs_data:
        all_suburbs.append(suburb_data['suburb_name'])
    suburbs_data_with_weather = []
    for suburb_name in all_suburbs:
        crime_weather_data = get_crime_weather_data(suburb_name)
        try:
            if crime_weather_data['average_temp'] is not None:
                suburbs_data_with_weather.append(crime_weather_data)
        except KeyError:
            pass
    return suburbs_data_with_weather

suburb_names_with_weather = get_suburbs_with_weather()

def get_by_total_offences(suburbs_data_with_weather):
    return sorted(suburbs_data_with_weather, key=lambda x: x['total_offences'], reverse=True)

def get_by_total_a_offences(suburbs_data_with_weather):
    return sorted(suburbs_data_with_weather, key=lambda x: x['total_a_offences'], reverse=True)

def get_by_total_d_offences(suburbs_data_with_weather):
    return sorted(suburbs_data_with_weather, key=lambda x: x['total_d_offences'], reverse=True)

In [9]:
# Button and Button Click Handler
def handle_s1_click(button,plot_output):
    data = get_by_total_offences(suburb_names_with_weather)[:10]
    suburbs = [entry['suburb_name'].upper() for entry in data]
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_map('crime',suburbs)
        plot_scatter_chart('average_temp',data)

def handle_s2_click(button,plot_output):
    data = get_by_total_offences(suburb_names_with_weather)[:10]
    suburbs = [entry['suburb_name'].upper() for entry in data]
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_map('crime',suburbs)
        plot_scatter_chart('average_rain',data)

def handle_s3_click(button,plot_output):
    data = get_by_total_offences(suburb_names_with_weather)[:10]
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_bar_chart('average_temp',data)

def handle_s4_click(button,plot_output):
    data = get_by_total_offences(suburb_names_with_weather)[:10]
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_bar_chart('average_rain',data)
        
def handle_s5_click(button,plot_output):
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_pie_chart(pie_data)

In [10]:
guide = """
    <b>Welcome to the Victoria Crime Analysis System!</b>
    <br>
    In light of the tragic knife attack at Westfield Bondi Junction in Sydney, there's been a renewed focus across Australia on developing effective crime prevention strategies.
    Both citizens and authorities are actively seeking effective solutions to prevent such alarming events. 
    The <b>Victoria Crime Analysis System</b> is your one-stop shop for data-powered 
    insights into crime trends within Victoria. We leverage big data analytics to uncover factors influencing criminal activity in urban areas.
    <hr>
    🔍 Simply click the search button next to your query, and our system will deliver a real-time response! 
    <hr>
"""
queries = [
    ("<b>Query 1: Do weather patterns influence the number of crimes?</b>", [
        ('1.1. Does temperature influence the number of crimes?', handle_s1_click),
        ('1.2. Does precipitation influence the number of crimes?', handle_s2_click)
    ]),
    ('<b>Query 2: Do weather patterns influence types of criminal incidents?</b>', [
        ('2.1. Does temperature affect the types of crimes?', handle_s3_click),
        ('2.2. Does precipitation affect the types of crimes?', handle_s4_click)
    ]),
    ('<b>Query 3: Is public sentiment correlated to number of criminal incidents?</b>', [
        ('3.1. Is public sentiment correlated to number of criminal incidents?', handle_s5_click)
    ])
]

# Widgets for display
plot_output = widgets.Output()
guide_html = widgets.HTML(guide)

# Create the rows for each query and sub-query
rows = [guide_html]

for query, sub_queries in queries:
    query_html = widgets.HTML(value=query)
    rows.append(query_html)
    for sub_query, handler in sub_queries:
        sub_query_html = widgets.HTML(value=sub_query)
        button = widgets.Button(description="Search")
        button.on_click(lambda button, h=handler: h(button, plot_output))
        rows.append(widgets.HBox(children=[sub_query_html, button]))

# Display Container
container = widgets.VBox(children=rows + [plot_output])
display(container)